In [2]:
import numpy as np

In [95]:
def file_to_differential(path):
    s = ""
    with open(path, 'r') as reader:
        s = reader.read()[10:-3]
    
    rows = []
    cols = []
    vals = []
    
    bitmask = np.int64(2**32)
    
    for elem in s.split(", "):
        n = np.int64(elem)
        rows.append((abs(n) // bitmask) - 1)
        cols.append((abs(n) % bitmask) - 1)
        vals.append(1 if n > 0 else -1)
    
    return vals, (rows, cols)

file_to_differential("/home/jay/desktop/qcat/KhoHo/differentials/knot_3_1/knot_3_1__d_-1_-3")

([1, 1, -1, -1, 1, 1], ([0, 1, 0, 1, 0, 1], [0, 0, 1, 1, 2, 2]))

In [117]:
import scipy
import scipy.sparse
import scipy.sparse.linalg

# compute laplacian from differentials
# def get_laplacian(d_i_minus_1, d_i):
    
#     if d_i != None:
#         D_i = d_i.transpose() * d_i

#     if d_i_minus_1 != None
#         D_i_minus_1 = d_i_minus_1 * d_i_minus_1.transpose()

#     return None

def get_eigs(laplacian):
    # get smallest eigenpair

    eig_val_small = -1
    if laplacian.det() != 0: # @TODO: somehow decide if this laplacian is nonsingular
        eig_val_small, eig_vec_small = scipy.sparse.linalg.eigsh(laplacian, k=1, sigma=0, maxiter=1000)
    
    eig_val_big, eig_vec_big = scipy.sparse.linalg.eigsh(laplacian, k=1, maxiter=1000)

    return (eig_val_small, eig_val_big)


In [104]:
import glob

def read_differential_files(dir):
    files = glob.glob(dir + "/*")

    dict = {}

    for file in files:
        data = file.split("_")
        i = int(data[-2])
        j = int(data[-1])
        
        dict[(i,j)] = file
    
    return dict

In [122]:

def get_diff_matrix(path):
    return scipy.sparse.coo_matrix(file_to_differential(path)).asfptype()

def get_knot_eigs(dir):
    dict = read_differential_files("./KhoHo/differentials/knot_3_1")
    keys = dict.keys()

    laplacians = {}

    for (i, j) in sorted(keys, reverse=True):

        d_i = get_diff_matrix(dict[(i,j)])

        laplacian = 0
        if (i-1,j) in keys:
            d_i_minus_1 = get_diff_matrix(dict[(i-1,j)])
            laplacian = (d_i_minus_1 * d_i_minus_1.transpose()) + (d_i.transpose() * d_i)
            laplacians[(i,j)] = laplacian

        else:
            laplacian = d_i.transpose() * d_i    
            laplacians[(i,j)] = laplacian

        if laplacian.nnz == 0:
            print("no non zero")

        if (i+1,j) not in laplacians:
            laplacian = d_i * d_i.transpose()

            if laplacian.nnz == 0:
                print("no non zero")
            
            laplacians[i+1,j] = laplacian
    
    with open("./eigs/knot_3_1_eigs", "w+") as writer:
        for (i,j) in laplacians.keys():

            if laplacians[(i,j)].shape == (1,1):
                # output_line = str(i) + " " + str(j) + " " + str(eig_small) + str(eig_large)
                # writer.write(output_line)
                continue
            
            eig_small, eig_large = get_eigs(laplacians[(i,j)])

            output_line = str(i) + " " + str(j) + " " + str(eig_small) + " " + str(eig_large) + "\n"
            writer.write(output_line)

get_knot_eigs("")
